In [ ]:
from collections import defaultdict, Counter
import random

from loguru import logger

from camel_up.game import Camel, CamelUpGame, Player
from camel_up.strategies import AlwaysRollStrategy, TakeLeaderBetSlipStrategy, RandomActionStrategy

In [ ]:
NUMBER_OF_GAMES = 100_000

In [ ]:
%%capture

# Redirects loguru to stderr so that it is captured, otherwise the Jupyter notebooks crashes.
import sys
logger.remove()
logger.add(sys.stderr)

seed_value = 42
random.seed(seed_value)

player_scores = defaultdict(list)
winners = []


for _ in range(NUMBER_OF_GAMES):

    players: list[Player] = [
        Player(AlwaysRollStrategy(), player_number=0), Player(AlwaysRollStrategy(), player_number=1), Player(AlwaysRollStrategy(), player_number=2), Player(TakeLeaderBetSlipStrategy(), player_number=3)
    ]
    camels: list[Camel] = [
        Camel(color=color) for color in ["red", "blue", "green", "purple", "yellow"]
    ]
    game: CamelUpGame = CamelUpGame(camels, players)
    while not game.is_game_finished():
        game.run_leg()

    winner = None
    winner_score = 0
    for player in game.players:
        player_scores[player.player_number].append(player.coins)
        if player.coins > winner_score:
            winner = player.player_number
            winner_score = player.coins

    winners.append(winner)

winners_count = Counter(winners)
winners
    

In [ ]:
# Player 0
winners_count[0]/400_000

In [ ]:
# Player 3
winners_count[3] / 400_000

In [ ]:
from plotnine import ggplot, aes, geom_histogram, theme_minimal, labs, geom_bar, ggtitle, ggsave, scale_y_continuous
import pandas as pd
from mizani.formatters import percent_format

strats = {**{i: "AlwaysRollDice" for i in range(2)}, **{3:"TakeLeaderBetSlip"}}
strat_data = []

for winner in winners:
    strat_data.append({'player_number': winner, 'player_strategy': strats[winner]})
df_legs = pd.DataFrame(strat_data)        

plot = (
    ggplot(df_legs, aes(x='player_strategy' )) 
    + geom_bar(stat='count',position='dodge',color='black', fill='blue')
    + labs(x="Player Strategy", y="Proportion")
    + ggtitle("Win Rate (3 RollDice, 1 TakeLeader)")
    + scale_y_continuous(labels=lambda l: ["%d%%" % (v * 100 / 100_000) for v in l])
    + theme_minimal()
)
ggsave(plot, filename='plots/2_3rolldice_1takeleader.png', dpi=300, width=8, height=6, units='in')
plot

In [ ]:
%%capture

# Redirects loguru to stderr so that it is captured, otherwise the Jupyter notebooks crashes.
import sys
logger.remove()
logger.add(sys.stderr)

seed_value = 42
random.seed(seed_value)

player_scores = defaultdict(list)
winners_3tl_1roll = []


for _ in range(NUMBER_OF_GAMES):

    players: list[Player] = [
        Player(TakeLeaderBetSlipStrategy(), player_number=0), Player(TakeLeaderBetSlipStrategy(), player_number=1), Player(AlwaysRollStrategy(), player_number=2), Player(TakeLeaderBetSlipStrategy(), player_number=3)
    ]
    camels: list[Camel] = [
        Camel(color=color) for color in ["red", "blue", "green", "purple", "yellow"]
    ]
    game: CamelUpGame = CamelUpGame(camels, players)
    while not game.is_game_finished():
        game.run_leg()

    winner = None
    winner_score = 0
    for player in game.players:
        player_scores[player.player_number].append(player.coins)
        if player.coins > winner_score:
            winner = player.player_number
            winner_score = player.coins

    winners_3tl_1roll.append(winner)

winners_3tl_1roll_count = Counter(winners)
winners_3tl_1roll
    

In [ ]:
from plotnine import ggplot, aes, geom_histogram, theme_minimal, labs, geom_bar, ggtitle, ggsave
import pandas as pd

strats = {**{i: "TakeLeaderBetSlip" for i in [0,1,3]}, **{2:"TakeLeaderBetSlip"}}
strat_data = []

for winner in winners_3tl_1roll:
    strat_data.append({'player_number': winner, 'player_strategy': strats[winner]})
df_legs = pd.DataFrame(strat_data)        

plot = (
    ggplot(df_legs, aes(x='player_strategy' )) 
    + geom_bar(stat='count',position='dodge',color='black', fill='blue')
    + labs(x="Player Strategy", y="Proportion")
    + ggtitle("Win Rate (1 RollDice, 3 TakeLeader)")
    + scale_y_continuous(labels=lambda l: ["%d%%" % (v * 100 / 100_000) for v in l])
    + theme_minimal()
)
ggsave(plot, filename='plots/2_1rolldice_3takeleader.png', dpi=300, width=8, height=6, units='in')
plot

In [ ]:
plot = (
    ggplot(df_legs, aes(x='factor(player_number)' )) 
    + geom_bar(stat='count',position='dodge',color='black', fill='blue')
    + labs(x="Player Strategy", y="Proportion")
    + ggtitle("Win Rate (1 RollDice, 3 TakeLeader, Player 2 = RollDice)")
    + scale_y_continuous(labels=lambda l: ["%d%%" % (v * 100 / 100_000) for v in l])
    + theme_minimal()
)
ggsave(plot, filename='plots/2_1rolldice_3takeleader_by_player.png', dpi=300, width=8, height=6, units='in')
plot